# 40. PYT. Feature Engineering

#### НАУЧИМСЯ:
1. удалять пропуски и дубли в данных;  
2. работать с датами и извлекать из них признаки;
3. заполнять пропуски константой или с помощью произвольной функции;
4. создавать новые признаки на основе предыдущих;
5. применять к признакам методы подсчёта статистик (например, среднего);
6. собирать разрозненные данные в единый датасет (при условии, что у них есть общие признаки);
7. группировать данные по какому-либо признаку и анализировать их.

In [1]:
import pandas as pd

In [7]:
import numpy as np

In [12]:
"""missingno предоставляет небольшой набор гибких и удобных в использовании отсутствующих 
визуализаций данных и утилит, которые позволяют быстро получить визуальное резюме полноты 
(или отсутствия) вашего набора данных"""

import missingno as msno

In [3]:
sample = pd.read_csv('data\sample.csv')
sample.columns  = ['name','city','age','profession']

In [35]:
log = pd.read_csv('data\log.csv', header = None)
log.columns  = ['user_id','time','bet','win'] # переименование столбцов

In [4]:
users = pd.read_csv('C:\\Users\\Sergey\\Desktop\\Python\\__pycache__\\modul_35\\data\\users.csv', sep="\t", encoding="koi8_r")
users.columns  = ['user_id','email','geo'] # переименование столбцов

### ЦЕЛЬ
Выяснить, какие регионы наиболее привлекательны для открытия офлайн-контор.

### ЗАДАЧИ
1. Сколько человеку надо прийти раз, чтобы сделать ставку?
2. Сколько в среднем составляет выигрыш?
3. Какой баланс по каждому пользователю?
4. В каких городах прибыль самая большая? 
5. В каких городах самая высокая ставка?
6. Сколько в среднем времени проходит с первого посещения сайта до первой попытки?

### КОНКРЕТНЫЕ ШАГИ (ФОРМАЛИЗОВАННЫЕ ЗАДАЧИ)

1. Выяснить, с какого по счёту визита посетитель делает ставку.
2. Определить средний выигрыш в процентах.
3. Рассчитать баланс по каждому пользователю.
4. Выявить самые выгодные города.
5. Определить города с самыми высокими ставками.
6. Выяснить, сколько в среднем времени разделяет первое посещение сайта и первую ставку.

## 40.2 Пропущенные значения

#### дополнительные параметры read_csv.
1. header=None — загрузить без строки с заголовком;
2. skiprows=n — пропустить n строк (часто у документов бывает техническая «шапка»);
3. encoding= — загрузить в конкретной кодировке;
4. na_values — список значений, который нужно заменить на NaN (специальный объект, обозначающий пропущенное значение).
5. sep=';' — разделитель

#### Какие бывают пропуски?
Список значений, которые по умолчанию считаются как пропуски: '', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', 'N/A', 'NA', 'NULL', 'NaN', 'n/a', 'nan', 'null'.

На практике чаще всего вы будете встречать '', 'NaN', 'nan', 'null'.

#### ПОИСК ПРОПУСКОВ
В Pandas есть метод isna(), который возвращает таблицу такой же размерности, что и на вход, но значения в ней — True или False (True, если данное значение является пропуском, и False — в ином случае).

Метод isna() есть не только у DataFrame, но и у Series. Это значит, что применять его можно не только ко всей таблице, но и к каждому столбцу отдельно.

In [5]:
log.head().isna() 

,user_id,time,bet,win
0,False,False,True,True
1,False,False,True,True
2,False,False,True,True
3,False,False,True,True
4,False,False,True,True


In [ ]:
users = pd.read_csv('C:\\Users\\Sergey\\Desktop\\Python\\__pycache__\\modul_35\\data\\users.csv', sep="\t", encoding="koi8_r")
users.columns  = ['user_id','email','geo'] # переименование столбцов

In [19]:
"""Сколько пропусков в столбце time?"""

log.time.isna().sum()

15

#### УДАЛЕНИЕ ПРОПУСКОВ
Удалять данные с пропусками можно с помощью метода dropna().

параметры метода:
1. axis - по какой оси удалять значения,
2. subset - Если передать в него список значений по одной оси (например, названия столбцов) и задать при этом в параметре axis другую ось (в нашем случае 0), то мы удалим те строки, для которых в данных столбцах находится пропуск.

In [ ]:
"""удаление строк с пропусками"""
log.dropna(axis=0)

In [ ]:
"""удаление столбцов с пропусками"""
log.dropna(axis=0)

In [34]:
"""Проверьте, есть ли пропуски в столбцах user_id и time. Если в столбце есть пропуски, 
удалите его. Сколько столбцов осталось в данных после этого?"""
log_new = pd.concat([log[['user_id', 'time']].dropna(axis=1), log[['bet', 'win']]], axis=1)
log_new 

,user_id,bet,win
0,Запись пользователя № - user_919,NaN,NaN
1,Запись пользователя № - user_973,NaN,NaN
2,Запись пользователя № - user_903,NaN,NaN
3,Запись пользователя № - user_954,NaN,NaN
4,Запись пользователя № - user_954,NaN,NaN
...,...,...,...
995,Запись пользователя № - user_984,9754.0,NaN
996,#error,10054.0,29265.0
997,#error,10454.0,NaN
998,#error,1000.0,NaN


## 40.3 Дубли

#### УДАЛЕНИЕ ПРОСТЫХ ДУБЛЕЙ
метод для удаления дублей — drop_duplicates()

У этого метода тоже есть параметр subset, но в этом случае ему нужно передавать список названий столбцов, а не их номера.


In [36]:
"""Удалите дубли среди столбцов user_id и time. Сколько строк осталось после этого?"""

log.drop_duplicates(subset=['user_id', 'time'])

,user_id,time,bet,win
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,NaN,NaN
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN
3,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,NaN
4,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,NaN
...,...,...,...,...
991,Запись пользователя № - user_965,[2019-04-20 12:55:41,800.0,6927.0
992,Запись пользователя № - user_967,[2019-04-20 14:59:36,10154.0,NaN
993,Запись пользователя № - user_973,[2019-04-20 17:09:56,10254.0,NaN
994,Запись пользователя № - user_977,[2019-04-20 18:10:07,10354.0,NaN


## 40.4 Преобразование в datetime

In [49]:
log = pd.read_csv('data\log.csv', header = None) # чтение файла
log.columns  = ['user_id','time','bet','win'] # переименование столбцов
log.time = log.time.str.strip('[') # убираем лишний символ
log.dropna(subset= ['time'], how='any', inplace=True) # убираем пропуски
log.time = pd.to_datetime(log.time) # преобразуем в формат даты

str(log.time.max())[:10]

'2019-04-20'

In [60]:
log = pd.read_csv('data\log.csv', header = None) # чтение файла
log = log.dropna() # убираем дубликаты
log.columns = ["user_id", "time", "bet", "win"] # переименование столбцов
log["time"] = log["time"].apply(lambda x: x[1:]) # убираем лишний символ
log["time"] = pd.to_datetime(log["time"])  # преобразуем в формат даты
log.time = log.time.apply(lambda x: x.minute)  # выбираем минуты
log.time.value_counts() # Какая минута встречалась в данных чаще всего?
log["time"].head()

14     57
29     59
151    54
189    34
205    26
Name: time, dtype: int64

In [ ]:
users = pd.read_csv('C:\\Users\\Sergey\\Desktop\\Python\\__pycache__\\modul_35\\data\\users.csv', sep="\t", encoding="koi8_r")
users.columns  = ['user_id','email','geo'] # переименование столбцов

## 40.5 Извлечение признаков времени

#### АТРИБУТЫ
1. asm8 - сжатый формат datetime64 в наносекундах
2. dayofweek - день недели
3. dayofyear - день в году
4. daysinmonth - день месяца
5. freqstr - общее кол-во дней в месяце
6. is_leap_year - если високосный год (true)
7. is_month_end - если последний день месяца (true)
8. is_month_start - если первый день месяца (true)
9. is_quarter_end - если последний день квартала (true)
10. is_quarter_start - если первый день квартала (true)
11. is_year_end - если последний день года (true)
12. is_year_start - если первый день года (true)
13. quarter - квартал
14. week - номер недели года

In [55]:
log = pd.read_csv('data\log.csv', header = None) # чтение файла
log = log.dropna() # убираем пустые значения
log.columns = ["user_id", "time", "bet", "win"] # переименование столбцов
log["time"] = log["time"].apply(lambda x: x[1:]) # убираем лишний символ
log["time"] = pd.to_datetime(log["time"])  # преобразуем в формат даты

In [57]:
"""мы хотим получить столбец, в котором каждым значением будет год из другого столбца. 
Это и есть Feature Engineering — создание новых признаков из старых."""
# Или
year_column = log["time"].apply(lambda x: x.year)
# или
year_column = log["time"].dt.year

value_counts()
метод возвращает серию (Series), которая содержит количество уникальных элементов, возвращает значения отсортированными по убыванию.

value_counts(ascending=True), отсортированными по возрастанию.

In [104]:
"""Какой месяц встречается в данных реже всего?"""

log = pd.read_csv('data\log.csv', header = None) # чтение файла
log.columns = ["user_id", "time", "bet", "win"] # переименование столбцов
time= log['time'].dropna() # столбец со временем с удалением пустых значений
time = time.apply(lambda x: x[1:]) # убираем лишний символ
time = pd.to_datetime(time)  # преобразуем в формат даты
time = time.apply(lambda x: x.month)  # выбираем месяц
time.value_counts(ascending=True) # Какой месяц встречается в данных реже всего?


4    170
2    259
3    264
1    292
Name: time, dtype: int64

In [108]:
"""Сколько дней в данных являются выходными (то есть суббота или воскресенье)?"""

log = pd.read_csv('data\log.csv', header = None) # чтение файла
log.columns = ["user_id", "time", "bet", "win"] # переименование столбцов
time= log['time'].dropna() # столбец со временем с удалением пустых значений
time = time.apply(lambda x: x[1:]) # убираем лишний символ
time = pd.to_datetime(time)  # преобразуем в формат даты
time = time.apply(lambda x: x.dayofweek)  # выбираем месяц
len(time[(time == 5) | (time == 6)]) # количество элементов в отфильтрованном списке

283

In [115]:
"""Какое время суток встречается в данных реже всего? Договоримся, что с 0 до 5 часов — ночь, 
с 6 до 11 — утро, с 12 до 17 — день, с 18 до 23 — вечер."""

log = pd.read_csv('data\log.csv', header = None) # чтение файла
log.columns = ["user_id", "time", "bet", "win"] # переименование столбцов
time= log['time'].dropna() # столбец со временем с удалением пустых значений
time = time.apply(lambda x: x[1:]) # убираем лишний символ
time = pd.to_datetime(time)  # преобразуем в формат даты
time = time.apply(lambda x: x.hour)
def time_day(x):
    if x <= 5:
        return 'ночь'
    elif x <= 11:
        return 'утро'
    elif x <= 17:
        return 'день' 
    else:   
        return 'вечер' 
    
time = time.apply(time_day) 
time.value_counts(ascending=True)

вечер    227
день     240
утро     253
ночь     265
Name: time, dtype: int64

In [117]:
"""Напишите код, который создаст признак hour из признака time в датасете log.csv. 
Для этого сделайте следующее:
    Загрузите датасет log.csv в переменную log, дальше работать будем с ней.
    Установите имена столбцов: ['user_id', 'time', 'bet', 'win']
    Избавьтесь от пропусков в log.
    Приведите переменную time к подходящему для извлечения признаков виду.
    Получите значение часа для каждой строки в переменной time и запишите в столбец hour в log.
    Результатом будет таблица log со столбцом hour внутри."""
    
log = pd.read_csv('data\log.csv', header = None) # чтение файла
log.columns = ["user_id", "time", "bet", "win"] # переименование столбцов
log = log.dropna() # убираем пустые значения
log['time'] = log['time'].apply(lambda x: x[1:]) # убираем лишний символ
log['time'] = pd.to_datetime(log['time'])  # преобразуем в формат даты
log['hour']  = log['time'] .apply(lambda x: x.hour)

log

    

,user_id,time,bet,win,hour
14,Запись пользователя № - user_917,2019-01-02 08:57:36,145732.0,1987653.0,8
29,Запись пользователя № - user_942,2019-01-04 13:59:42,1678321.0,9876543.0,13
151,Запись пользователя № - user_982,2019-01-16 21:54:22,100.0,4749.0,21
189,Запись пользователя № - user_964,2019-01-21 18:34:44,200.0,4667.0,18
205,Запись пользователя № - user_931,2019-01-22 05:26:59,300.0,4319.0,5
...,...,...,...,...,...
967,Запись пользователя № - user_975,2019-04-19 22:25:15,1000.0,6108.0,22
971,Запись пользователя № - user_912,2019-04-20 10:35:49,10554.0,31799.0,10
972,Запись пользователя № - user_926,2019-04-20 10:35:50,10354.0,30244.0,10
976,Запись пользователя № - user_970,2019-04-20 10:35:54,10354.0,30691.0,10


## 40.6 И снова о пропусках

#### ЗАПОЛНЕНИЕ КОНСТАНТОЙ
Чтобы заполнить пропуски в столбце каким-то значением, можно использовать метод fillna() у самого столбца. Аргументом этого метода будет число, которое появится на месте пропусков

In [142]:
log = pd.read_csv('data\log.csv', header = None) # чтение файла
log.columns = ["user_id", "time", "bet", "win"] # переименование столбцов

In [128]:
log["bet"] = log["bet"].fillna(0)

In [130]:
"""Cколько раз люди приходили, но не делали ставки?"""
log[log["bet"] == 0]["user_id"].count()

515

In [131]:
log.win = log.win.fillna(0)

def fillna_win(row):
    # Нужно дописать  
    if not row['bet']:
        return 0
    else:
        if row['win']:
            return row['win']
        else:
            return -row['bet']
  
# Применяем функцию  
new_win = log.apply(lambda row: fillna_win(row), axis=1)  
  
# Заменяем старый столбец с пропусками на новый без пропусков  
log['win'] = new_win 

In [132]:
"""Сколько раз участники ставок проиграли деньги?"""
log[log["win"] < 0]["user_id"].count()

347

In [ ]:
users = pd.read_csv('C:\\Users\\Sergey\\Desktop\\Python\\__pycache__\\modul_35\\data\\users.csv', sep="\t", encoding="koi8_r")
users.columns  = ['user_id','email','geo'] # переименование столбцов

In [137]:
log['net'] = log["win"] - log["bet"] # здесь должен быть код, создающий новый признак  
log

,user_id,time,bet,win,net
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,0.0,0.0,0.0
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,0.0,0.0,0.0
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,0.0,0.0,0.0
3,Запись пользователя № - user_954,[2019-01-01 17:17:51,0.0,0.0,0.0
4,Запись пользователя № - user_954,[2019-01-01 21:31:18,0.0,0.0,0.0
...,...,...,...,...,...
995,Запись пользователя № - user_984,[2019-04-20 9:59:58,9754.0,-9754.0,-19508.0
996,#error,NaN,10054.0,29265.0,19211.0
997,#error,NaN,10454.0,-10454.0,-20908.0
998,#error,NaN,1000.0,-1000.0,-2000.0


In [138]:
"""У какого количества людей выигрыш положительный? Для ответа на вопрос создайте признак 
net, хранящий сумму выигрыша с учётом ставки."""

log[log["net"] > 0]["user_id"].count()

138

In [139]:
"""Каково среднее значение выигрыша (из столбца net) в тех случаях, когда выигрыш больше 0?
Введите ответ, не округляя, но отбросив дробную часть."""

log[log["net"] > 0]["net"].mean()


80253.33333333333

In [141]:
"""Каково медианное значение выигрыша (из столбца net) в тех случаях, когда выигрыш больше 0?"""

log[log["net"] > 0]["net"].median()

5347.0

In [146]:
"""Как можно посчитать среднее значение для столбца bet, не учитывая при подсчёте пропуски?
4 варианта:"""

log.bet.mean()
log.bet.sum() / log.bet.dropna().shape[0]
np.mean(log.bet)
log['bet'].dropna().mean()


6785.738144329897

In [155]:
"""Выполните следующие действия:
   Замените пропущенные значения в столбцах bet и win таблицы log.csv на 0.
   Создайте столбец net, хранящий сумму выигрыша с учётом ставки (для этого из столбца 
win поэлементно вычтите столбец bet и запишите в новый столбец).
   Посчитайте, какой процент посещений букмекерской конторы оборачивался ставкой. Для этого
   поделите количество ставок (значений больше 0) на общее количество посещений конторы."""
   
log = pd.read_csv('data\log.csv', header = None) # чтение файла
log.columns = ["user_id", "time", "bet", "win"] # переименование столбцов   
log["bet"] = log["bet"].fillna(0) # замена пропусков
log["win"] = log["win"].fillna(0) # замена пропусков
log['net'] = log["win"] - log["bet"]

In [ ]:
log = pd.read_csv('data\log.csv', header = None) # чтение файла
log.columns = ["user_id", "time", "bet", "win"] # переименование столбцов

In [157]:
"""Посчитайте, какой процент посещений букмекерской конторы оборачивался ставкой. 
Для этого поделите количество ставок (значений больше 0) на общее количество посещений 
конторы."""

log[log["bet"] > 0]["user_id"].count() / log["user_id"].count() * 100

48.5

In [158]:
"""Каково среднее значение ставки (из столбца bet) в тех случаях, когда ставка была сделана?"""

log[log["bet"] > 0]["bet"].mean()

6785.738144329897

In [161]:
"""Каков средний выигрыш (из столбца net) в тех случаях, когда ставка была сделана?
Выигрышем будем считать любое изменение количества денег, в том числе отрицательное 
(в таком случае это проигрыш)."""

log[(log["bet"] > 0) & (log["net"] != 0)]["net"].mean()

20421.892783505155

In [167]:
"""Каков средний размер потерь при проигрыше (из столбца net)?"""

log[log["net"] < 0]["net"].mean()

-3372.743515850144

In [166]:
"""Чем чаще всего заканчивается ставка — выигрышем или проигрышем?"""

a = round(log[log["net"] < 0]["net"].count()/log["user_id"].count() * 100)
b = round(log[log["net"] > 0]["net"].count()/log["user_id"].count() * 100)
print(a,b)

35 14


In [171]:
"""Напишите код, который узнает, чему была равна минимальная ставка и сколько людей сделали такую ставку. Для этого:
    1. Загрузите датасет log.csv.
    2. Посчитайте, чему равна минимальная ставка.
    3. Посчитайте, сколько раз была сделана минимальная ставка и запишите результат в 
    переменную min_bet_amount в виде целого числа."""
    
log = pd.read_csv('data\log.csv', header = None) # чтение файла
log.columns = ["user_id", "time", "bet", "win"] # переименование столбцов    
min_bet = log["bet"].min()
min_bet_amount = log[log["bet"] == min_bet]["user_id"].count() 
min_bet_amount

numpy.int32

## 40.9 Объединяем датасеты

In [205]:
"""Повторим часть предобработки, которую мы должны были сделать ранее:"""

log = pd.read_csv('data\log.csv', header = None) # чтение файла
log.columns = ["user_id", "time", "bet", "win"] # переименование столбцов   
# Приведём признак user_id к одному формату в обоих датасетах
users.user_id = users.user_id.apply(lambda x: x.lower())
# Избавимся от ошибок в user_id
log = log[log.user_id != "#error"]
log.user_id = log.user_id.str.split(" - ").apply(lambda x: x[1])

In [206]:
"""Теперь объединим данные с помощью метода pd.merge():"""

merged = pd.merge(log, users, on='user_id')  

In [207]:
"""В данном случае мы группируем данные по признаку user_id.
   После этого в каждой группе выбираем признак win.
   Затем берём медиану по каждой группе по признаку win и на выходе получаем таблицу, 
где индексом является признак user_id. В этой таблице единственный столбец — медиана по 
каждой группе (то есть по каждому пользователю).
   Наконец, последний вызов median() даёт нам медиану по предыдущему столбцу, то есть 
возвращает одно число."""

merged.groupby('user_id').win.median().median() 

5951.75

## 40.10 Анализ данных

In [251]:
log = pd.read_csv('data\log.csv', header = None) # чтение файла
log.columns = ["user_id", "time", "bet", "win"] # переименование столбцов   
log["bet"] = log["bet"].fillna(0) # замена пропусков
log["win"] = log["win"].fillna(0) # замена пропусков
log['net'] = log["win"] - log["bet"]
users.user_id = users.user_id.apply(lambda x: x.lower()) # Приведём признак user_id к одному формату в обоих датасетах
log = log[log.user_id != "#error"] # Избавимся от ошибок в user_id
log.user_id = log.user_id.str.split(" - ").apply(lambda x: x[1])
log['time'] = log['time'].apply(lambda x: x[1:]) # убираем лишний символ
log['time'] = pd.to_datetime(log['time'])  # преобразуем в формат даты
merged = pd.merge(log, users, on='user_id')  

In [215]:
"""Какова медиана баланса по каждому пользователю?"""

merged.groupby('user_id').net.sum().median()

1986.0

In [216]:
"""Сколько раз в среднем каждый человек приходит, не делая ставок, при условии, что у этого 
человека все-таки есть хотя бы одна ставка?
Просуммируйте в каждой группе количество записей со ставкой, равной 0, и поделите на 
общее количество групп. Если при этом в группе нет записей со ставкой больше 0, считаем 
количество записей в данной группе равным 0."""

merged[merged.bet == 0].user_id.count() / merged.user_id.nunique()

5.05

In [217]:
merged

,user_id,time,bet,win,net,email,geo
0,user_919,2019-01-01 14:06:51,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
1,user_919,2019-01-30 10:06:00,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
2,user_919,2019-02-05 14:33:44,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
3,user_919,2019-02-14 11:38:05,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
4,user_919,2019-03-02 04:23:36,300.0,0.0,-300.0,Chikkaverle@icloud.com,Хабаровск
...,...,...,...,...,...,...,...
970,user_932,2019-02-24 22:40:06,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск
971,user_932,2019-03-15 10:56:14,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск
972,user_932,2019-03-18 10:13:24,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск
973,user_932,2019-03-27 12:18:24,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск


In [244]:
"""Сколько времени в среднем проходит между появлением человека в сервисе и первой ставкой? 
Считать нужно только тех, кто делал ставку и у кого в файле зарегистрирован первый визит 
без ставки."""
"""Для ответа на вопрос напишите метод, считающий минимальное время среди ставок, равных 0, 
и минимальное время среди ставок больше 0. После этого верните разницу между вторым и первым
числом. Пройдитесь по всем группам. Если в группе нет ставок больше 0, пропустите эту группу. 
Просуммируйте разницу во времени для каждой группы (с помощью метода, описанного выше) и поделите 
на количество групп, которые вы не пропустили."""

a = merged[merged.bet == 0].groupby('user_id').time.min()
b = merged[merged.bet > 0].groupby('user_id').time.min()
c = pd.merge(a, b, on='user_id')  
c['day'] = c['time_y'] - c['time_x']
c['day'][c['day'] > pd.Timedelta(0)].mean()


Timedelta('49 days 13:01:38.873684210')

In [250]:
"""Наибольший суммарный выигрыш среди всех городов имеет Москва. Определите, какой город 
следует за ней."""

merged.groupby('geo').win.sum().sort_values(ascending=False)

geo
Москва             11959741.0
Воронеж              184338.0
Санкт-Петербург      151007.0
Казань                97806.0
Ярославль             97441.0
Ижевск                84895.0
Красноярск            84767.0
Арзангелтск           74375.0
Пермь                 67734.0
Хабаровск             65459.0
Краснодар             62718.0
Ставрополь            46003.0
Екатеринбург          36682.0
Тюмень                 4701.0
Name: win, dtype: float64

In [257]:
"""Во сколько раз различаются максимальное и минимальное значения средней ставки по городам?"""
a = merged[merged.bet > 0].groupby('geo').bet.mean().sort_values(ascending=False)
a.max()/a.min()

127.81602335164835

In [271]:
"""Напишите код, который посчитает, сколько раз пользователи приходили в букмекерскую контору в каждом городе. Для этого:
    1. Загрузите датасеты log.csv и users.csv.
    2. Удалите значения из user_id с ошибкой (#error) и приведите признак user_id к 
одному виду в обоих датасетах.
    3. Слейте два датасета в один по признаку user_id.
    4. Сгруппируйте данные по правильному признаку (его вам нужно определить самостоятельно), 
затем выберите user_id и воспользуйтесь функцией count() для подсчёта наблюдений в каждой группе.
    5. Результат (таблицу) запишите в sample2."""
    
log = pd.read_csv('data\log.csv', header = None) # чтение файла
log.columns = ["user_id", "time", "bet", "win"] # переименование столбцов  
users = pd.read_csv('C:\\Users\\Sergey\\Desktop\\Python\\__pycache__\\modul_35\\data\\users.csv', sep="\t", encoding="koi8_r") 
users.columns  = ['user_id','email','geo'] # переименование столбцов
users.user_id = users.user_id.apply(lambda x: x.lower()) # Приведём признак user_id к одному формату в обоих датасетах
log = log[log.user_id != "#error"] # Избавимся от ошибок в user_id
log.user_id = log.user_id.str.split(" - ").apply(lambda x: x[1])
log['time'] = log['time'].apply(lambda x: x[1:]) # убираем лишний символ
log['time'] = pd.to_datetime(log['time'])  # преобразуем в формат даты
merged = pd.merge(log, users, on='user_id')  
sample2 = merged.groupby(['geo']).user_id.count()
sample2

    

geo
Арзангелтск         96
Воронеж             88
Екатеринбург        49
Ижевск              61
Казань              66
Краснодар           86
Красноярск          56
Москва              61
Пермь               55
Санкт-Петербург    115
Ставрополь          36
Тюмень              32
Хабаровск           60
Ярославль           89
Name: user_id, dtype: int64

#### ВЫВОД:
Реже всего люди заходят в букмекерскую контору вечером.

Почти в половине случаев люди приходили, но не делали ставок.

А те, кто делали ставки, проиграли больше чем в двух третях случаев.

Средняя ставка и средний проигрыш примерно равны по абсолютному значению.

Между первым приходом человека и первой ставкой может пройти довольно много времени. И за это время человек может зайти в контору ещё несколько раз, не делая ставок.

Забавный факт: второй по величине город России и второй по суммарному выигрышу на ставках город России — разные города.

И самое важное: мы поняли, что вы скорее проиграете, чем выиграете, поэтому лучше инвестировать в знания, а не ставки.

#### ВЫВОД ПО МОДУЛЮ:

работать с пропущенными значениями и дублями;

работать со временем: приводить строковые значения ко встроенному в Pandas типу времени, извлекать признаки из даты (порой нетривиальные);

анализировать табличные данные с целью поиска инсайтов, использовать для этого стандартные функции Pandas;

группировать данные и использовать вычисление статистик по группам для нахождения более глубокой информации в данных.